# Platform Positioning

In [1]:
from __future__ import print_function

In [2]:
import random as random #For random initial platform position
import numpy as np
import pandas as pd

import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point, LineString 
import shapely
from geopy import distance # For calculating distance between two points 


import pickle  as pkl # For importing pickled objects

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from math import tan, isclose, pi #F or calculating point on circumference - step out radius

# Ingest Processed Data
### Target Polygons
These were processed in the preceeding notebook and stored in '1_processed'

In [3]:
with open('data/1_processed/target_polygon_gdf.pkl', 'rb') as f:
    gdf_targPoly = pkl.load(f)
    
gdf_targPoly.plot()

### Well Concepts
As no suitable test data exists, dummy data is generated. There is a separate dummdatacreator.py file containing this work.

Rather than have Well Concepts randomly placed anywhere, we restrict it to only be within Target Polygon space

In [4]:
from dummydatacreator import  GenerateMultipleWellConceptsTEST as GenerateWCs # This is not supported in Azure notebooks

In [5]:
WC_name, WC_points, WC_resource = GenerateWCs(gdf_targPoly)

print("Well Concepts Generated:\n",'\n'.join('Name: {}, Points: {}, Resource: {}'.format(WC_name[k], WC_points[k], WC_resource[k]) for k in range(5)))

Well Concepts Generated:
 Name: C-296, Points: [-60.36349177635691, 10.048277308519275], Resource: 30.205184487599546
Name: Td-282, Points: [-60.31943385897276, 10.092420977450292], Resource: 31.212501337507803
Name: Z-384, Points: [-60.310296248575035, 10.089832362345108], Resource: 3.5278640046138934
Name: Ac-726, Points: [-60.3178199286206, 10.093650573602595], Resource: 12.008848146446256
Name: D-196, Points: [-60.31179207134214, 10.122276978013776], Resource: 1.9487037526992461


# Mapping
In order to construct the map containing Targets, Well Concepts, Platforms, some functions are required:


### Platform Step-Out Radius Functions
As there is no method for deriving the circumference around a point in EPSG:4326, functions are required to calculate this.
- A function to find a single point on the circumference (within a given tolerance)
- A function to find multiple points all around the circumference, sufficient for plotting the circle

In [6]:
def GetXYatCircumference(centre, desired_radius, theta, tolerance = 0.00001):
    '''Calculates the X,Y coordinates of a point on the circumference of a circle, 
    with a given desired radius and angle to the vertical.
    
    Theta in radians
    '''
    
    #Initialise x,y coordinates
    x = centre[0]
    y = centre[1]

    
    #Set initial increment values
        #If we are working in the lower two quadrants, i.e. where y will be less than centre y
    if theta > pi/2 and theta < (3*pi)/2:
        y_increment=-0.01
    else:
        y_increment=0.01

        
    #Loop until the radius of the point and centre, is close to the desired radius
    while isclose(distance.distance(centre, [x,y]).kilometers,  desired_radius ,rel_tol=0.00001) == False:

        # Add the increment whilst the distance is less than desired
        while distance.distance(centre, [x,y]) < desired_radius:
            x_increment = y_increment * tan(theta)
            x += x_increment
            y += y_increment

        # Subtract the increment whilst the distance is more than desired
        while distance.distance(centre, [x,y]) > desired_radius:
            x_increment = y_increment * tan(theta)
            x -= x_increment
            y -= y_increment

        #Reduce the size of the increment to increase precision  
        y_increment = y_increment/4

    return [x,y]

In [7]:
def GetCircumferenceXY(centre, radius, n_points, tolerance = 0.00001):
    '''Calculates x,y coordinates of a specified number points on the circumference of a 
    circle with a specified centre    
    
    NB: n_points must odd, to ensure we do not get a trig erro
    '''
    assert n_points % 2 != 0, "n_points must not be an even number - gives a trig error"

    # The angle between each radius        
    theta_increment = (2*pi)/n_points

    # The angles of all the radii we will use
    thetas = [theta_increment * i for i in range(n_points)]
    
    # Get a list of all calculated XYs on the circumference
    xys = [GetXYatCircumference(centre, radius,theta_i,tolerance) for theta_i in  thetas]
    
    # Create a full 'loop' so the first and last points are the same - Polygon
    xys.append(xys[0])
    
    
    xs = [xy[0] for xy in xys]
    ys = [xy[1] for xy in xys]
    
    return xs, ys

GetCircumferenceXY([0,0], 2, 3)

([0.0, 0.015637707705740515, -0.015637707705740488, 0.0],
 [0.01796615600585938,
  -0.009028434753417969,
  -0.009028434753417969,
  0.01796615600585938])

### Well Concept Relative Position
i.e. Are Well Concepts within specified radii around a point?

In [8]:
def BinByDistances(pointsToRank, centre, distBins):
    '''Well Concepts are ranked according to binned distance from platform. i.e. 
    Given input distances [0,2,5], the function will return a list of rankings where:
    0 indicates Well Concept is more than 5 km from platform
    1 indictates Well Concept is between 2 and 5 km from platform... etc
    '''
    #Radii must be ordered, i.e. [0,2,5]
    distBins = np.sort(distBins)
    
    num_bins = len(distBins)

    #calculate distances of all well concepts from platform 
    distances = [distance.distance(point_i, centre) for point_i in pointsToRank] #Karney 2013 distance
    
    distances = np.reshape(distances,[-1])

    
    #Reverse ranking - where 0 is closest to platform
    revranks = np.digitize(distances, distBins, right = False) #i.e. includes left bound = 0

    # Ranking - where 0 is furthest from platform
    ranking = [num_bins - revrank for revrank in revranks]
    
    return ranking

In [9]:
dct_colors = {0: "#808080",
          1:"#FF8000",
          2:"#00CC00"}

In [10]:
def GetWCRanks(wcpoints, plat_points,radii):
    '''
    Given the rank of Well Concepts, this derives the appropriate color for them
    
    If a WC has a rank of 3 (i.e. very close) with a platform, but a rank of 0 (i.e. far) from another platform, we 
    only care about the ranking of 3
    '''
    #Subset the WCs to only those which are on the creaming curve
    #A list for each platform, in which each WC is given a rank/bin
    WC_rank_by_platform = []
    
    for plat_point in plat_points:
        WC_rank_by_platform.append(BinByDistances([(a[0], a[1]) for a in wcpoints],plat_point,radii) )

    WC_best_rank = [max(WC_ranks)   for WC_ranks in zip(*WC_rank_by_platform)]
    
    
    return WC_best_rank  

## Map Construction

https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9

In [19]:
import json
from bokeh.models.widgets import Slider

from bokeh.io import output_file, show, save

from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool, LabelSet,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
# Input GeoJSON source that contains features for plotting
from bokeh.models.markers import SquareCross
from bokeh.models.callbacks import CustomJS


from bokeh.layouts import column, row, WidgetBox
from bokeh.models import Panel
from bokeh.models.widgets import Tabs

In [12]:
lon_min = -60.64
lon_max = -60.1
lat_min = 9.85
lat_max = 10.5
n_concepts = 150

WC_name, WC_points, WC_resource = GenerateWCs(gdf_targPoly, n_concepts, lon_min, lon_max, lat_min, lat_max)

#Defaults
radiusinner=1
radiusouter=1.1





#Any calculations based on the above defaults
n_plat = 20
_, plat_points, _ = GenerateWCs(gdf_targPoly, n_plat, lon_min, lon_max, lat_min, lat_max)
plat_points = list(plat_points)
plat_name = ["Platform" for i in range(len(plat_points))]


_ = GetWCRanks(WC_points,plat_points, [0,radiusinner,radiusouter])
WC_colors = [dct_colors[wcrank] for wcrank in _]  




innerstepoutsx = []
innerstepoutsy = []

for plat_point in plat_points:
    innerstepoutx, innerstepouty = GetCircumferenceXY(plat_point,radiusinner , 401)
    innerstepoutsx.append(np.array(innerstepoutx[:-1]))
    innerstepoutsy.append(np.array(innerstepouty[:-1]))

outerstepoutsx = []
outerstepoutsy = []

for plat_point in plat_points:
    outerstepoutx, outerstepouty = GetCircumferenceXY(plat_point,radiusouter , 401)
    outerstepoutsx.append(np.array(outerstepoutx[:-1]))
    outerstepoutsy.append(np.array(outerstepouty[:-1]))
    


In [13]:
plat_points

[[-60.5981771192637, 10.389360294798871],
 [-60.2046331466558, 10.065849674972547],
 [-60.422748918931305, 10.019408742458037],
 [-60.44363623547415, 9.890256728259692],
 [-60.14937512232509, 10.10708731326353],
 [-60.48142310713877, 9.884569274234826],
 [-60.52500293388781, 9.909059901838992],
 [-60.2616253547809, 10.064390723639207],
 [-60.30040158862338, 10.125185552723744],
 [-60.29251370313392, 10.10128489024894],
 [-60.506215718728704, 9.982453781864395],
 [-60.291811661840015, 10.122392924149457],
 [-60.41909015286366, 10.019856147119258],
 [-60.28989664726852, 10.117845230748365],
 [-60.49322266538776, 9.867179206750466],
 [-60.24204073019855, 10.102835492880912],
 [-60.31707860019149, 10.058492681889796],
 [-60.327935125394546, 10.068447456908292],
 [-60.46439855173611, 9.86067693273085],
 [-60.18877804017113, 10.08115273834424]]

### Bokeh Map

In [280]:


# Create figure object.
p = figure(title = 'Polygons', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset",
           active_scroll = "wheel_zoom",
            x_range=(lon_min, lon_max),
            y_range=(lat_min, lat_max),          
          )
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None



# TARGET LAYER #
# Data Source
ds_targ = GeoJSONDataSource(geojson = gdf_targPoly[['target_id', 'target_name', 'polygon']].to_json())

# Bokeh Layer
p_targ = p.patches('xs','ys', source = ds_targ,
                   fill_color = "lightskyblue",
                   line_color = "blue", 
                   line_width = 0.1, 
                   fill_alpha = 0.2)


#PLATFORM LAYER #
platx = [a[0] for a in plat_points]
platy = [a[1] for a in plat_points]


# Data Source 
ds_plat = ColumnDataSource({'x':platx, 'y':platy, 'name': plat_name})
# Bokeh Layer
p_plat = p.diamond_cross("x", "y", size=20, source=ds_plat,
         line_color="black", fill_alpha=0.8)
# Label layer
labels = LabelSet(x="x", y="y", text="name", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=ds_plat, text_align='center')
p.add_layout(labels)


#PLATFORM INNER STEP-OUT INNER #
#Data Source
source_step_inner = ColumnDataSource({'x':innerstepoutsx, 'y':innerstepoutsy})

#Bokeh layer
p_step_inner = p.patches('x','y', source = source_step_inner,
                   fill_color = None,
                   line_color = "#00CC00", 
                   line_width = 2, 
                   fill_alpha = 1)

#PLATFORM STEP-OUT OUTER #
source_step_outer = ColumnDataSource({'x':outerstepoutsx, 'y':outerstepoutsy})

p_step_outer = p.patches('x','y', source = source_step_outer,
                   fill_color = None,
                   line_color = "#FF8000", 
                   line_width = 1, 
                   fill_alpha = 0.1)

# WELL CONCEPTS
wcx = [a[0] for a in WC_points]
wcy = [a[1] for a in WC_points]
ds_wc = ColumnDataSource({'x':wcx, 'y':wcy, 'name':WC_name, 'resource':WC_resource, 'hex': WC_colors})

p_wc = p.circle("x", "y", size=5, source=ds_wc,
         color='hex', line_color="black", fill_alpha=0.8)

labels = LabelSet(x="x", y="y", text="name", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=ds_wc, text_align='center')
p.add_layout(labels)

# Create hover tool
p.add_tools(HoverTool(renderers = [p_wc],
                      tooltips = [('Well Concept','@name'),
                                ('Resource','@resource')]))

### MPL Bar Chart - Creaming Curve

In [281]:
#Subset the WCs to only those which are on the creaming curve
WC_rank = GetWCRanks([(a[0], a[1]) for a in WC_points],plat_points,[0,radiusinner, radiusouter])

#Construct dataframe of all well concepts
df_WC = pd.DataFrame({"Well Concept": WC_name,"Resource": WC_resource, "Rank":WC_rank, "x":[c[0] for c in WC_points]})

#Reduce to df of well concepts on Creaming Curve (i.e. within the radii)
df_WC_cc = df_WC[df_WC['Rank'] != 0].copy()

#Map ranks to colors
df_WC_cc.loc[:,'Color'] = df_WC_cc['Rank'].map(lambda x: dct_colors[x])

df_WC_cc.sort_values(by="Resource", inplace=True, ascending=False)

b = figure(x_range=df_WC_cc['Well Concept'], plot_height=250, title="Creaming Curve")
b.vbar(x=df_WC_cc['Well Concept'], top=df_WC_cc['Resource'], color=df_WC_cc['Color'], width=0.9)

b.xgrid.grid_line_color = None
b.y_range.start = 0

In [282]:
names, res, col, x = df_WC_cc[['Well Concept', 'Resource', 'Color', 'x']].T.values

dct = {'name':names.tolist(), 'resource':res.tolist(), 'color':col.tolist(), 'x':x}

cds_visibleWC = ColumnDataSource(dct)

d = figure(x_range=names, plot_height=250, title="Creaming Curve 2")

d.vbar(x='name', top='resource', color='color', source=cds_visibleWC, width=0.9)

GlyphRenderer(id='29599', ...)

### Create Bokeh Layout

In [283]:
output_file("p.html")

p.x_range.callback = CustomJS(
        args=dict(source=cds_visibleWC), code=
"""
var data = source.data;


let visiblewc = [];
let visibleres = [];
let visiblecol = [];
let visiblex = [];



console.log('init');

for (let i = 0; i < data.x.length; i++){

    if (data.x[i] > cb_obj.start && data.x[i] < cb_obj.end){
        visiblex.push(data.x[i]);
        visiblewc.push(data.name[i]);
        visibleres.push(data.resource[i]);
        visiblecol.push(data.color[i]);
        
        
        
        
    }
}

console.log(visiblex);
//callback object - i.e. x_range
data.x = visiblex;
data.name = visiblewc;
data.resource = visibleres;
data.col = visiblecol;



source.change.emit();
""")
# Create a row, with control panel and plot
layout = row(p, b, d)

# Make a tab with the layout 
tab = Panel(child=layout, title = 'Geospatial PoC')
tabs = Tabs(tabs=[tab])
save(tabs)

'/Users/Pri.balachandran@ibm.com/Desktop/BP/20191106_PlatformWCAnalysis/p.html'

In [17]:
WC_name

('I-638',
 'L-803',
 'I-709',
 'Rc-140',
 'Pc-646',
 'Sc-57',
 'Q-794',
 'L-431',
 'L-730',
 'J-13',
 'H-589',
 'L-51',
 'Y-191',
 'V-705',
 'C-116',
 'P-53',
 'R-382',
 'Yc-756',
 'G-192',
 'P-400',
 'Qd-804',
 'W-650',
 'Qd-600',
 'V-579',
 'Xc-406',
 'A-411',
 'Z-937',
 'W-251',
 'D-942',
 'J-925',
 'W-854',
 'R-264',
 'K-644',
 'V-185',
 'Hd-638',
 'Db-277',
 'M-356',
 'L-533',
 'B-618',
 'P-164',
 'I-603',
 'J-695',
 'Ad-891',
 'Ec-110',
 'L-309',
 'F-244',
 'M-997',
 'Z-287',
 'S-158',
 'X-867',
 'M-716',
 'P-929',
 'Eb-570',
 'J-724',
 'I-840',
 'Zc-219',
 'Gd-342',
 'G-264',
 'S-506',
 'X-557',
 'Ic-349',
 'X-352',
 'E-320',
 'W-455',
 'Xc-919',
 'Dc-574',
 'V-232',
 'K-568',
 'S-534',
 'Jb-843',
 'K-463',
 'Y-320',
 'I-837',
 'Y-557',
 'C-890',
 'N-11',
 'Z-609',
 'Ib-519',
 'Td-673',
 'B-291',
 'Kc-641',
 'Bc-525',
 'E-973',
 'Z-894',
 'Zd-550',
 'B-833',
 'S-129',
 'Q-649',
 'Zc-816',
 'H-416',
 'Y-274',
 'A-182',
 'W-229',
 'Q-197',
 'D-194',
 'Ec-429',
 'N-72',
 'Qd-990',


In [18]:
xs

NameError: name 'xs' is not defined

In [ ]:
stepoutsx[1]